## 우선 사용할 DB 생성

In [6]:
%load_ext sql

In [7]:
# ID와 PW를 자신의 환경에 맞게 수정
%sql postgresql://...

'Connected: seankkang@dev'

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS seankkang.name_gender (
   name varchar(32),
   gender varchar(8)
);


 * postgresql:/...
Done.


[]

In [1]:
import psycopg2

# Redshift connection 함수
def get_Redshift_connection():
    host = "..."
    redshift_user = "..."
    redshift_pass = "..."
    port = 5439
    dbname = "dev"
    conn = psycopg2.connect("dbname={dbname} user={user} host={host} password={password} port={port}".format(
        dbname=dbname,
        user=redshift_user,
        password=redshift_pass,
        host=host,
        port=port
    ))
    conn.set_session(autocommit=True)
    return conn.cursor()

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# ETL 함수를 하나씩 정의

In [2]:
import requests

def extract(url):
    f = requests.get(url)
    return (f.text)

In [3]:
def transform(text):
    lines = text.split("\n")
    return lines

In [8]:
def execute_with_log(cursor, sql):
  print(f'Execute : {sql};')
  cursor.execute(sql)

target_table = 'seankkang.name_gender'

def load(lines):
    # BEGIN과 END를 사용해서 SQL 결과를 트랜잭션으로 만들어주는 것이 좋음
    # BEGIN;TRUNCATE TABLE;INSERT INTO TABLE VALUES ('KEEYONG', 'MALE');END;
    cur = get_Redshift_connection()

    # Starg transaction
    execute_with_log(cur, f'BEGIN')
    # Truncate table before Insert 
    execute_with_log(cur, f'TRUNCATE TABLE {target_table}')

    for record in lines[1:]: # except header
      if record and record.strip():
        name, gender = record.split(',')
        print(f'{name} - {gender}')

        if name and gender:
          execute_with_log(cur, f"INSERT INTO {target_table} VALUES ('{name}', '{gender}')")
        
    # End transation 
    execute_with_log(cur, f'END')

# 이제 Extract부터 함수를 하나씩 실행

In [9]:
%%sql
SELECT COUNT(1) FROM seankkang.name_gender

 * postgresql://...
1 rows affected.


count
0


In [10]:
link = "https://s3-geospatial.s3-us-west-2.amazonaws.com/name_gender.csv"

data = extract(link)

In [11]:
lines = transform(data)

In [12]:
load(lines)

Execute : BEGIN;
Execute : TRUNCATE TABLE seankkang.name_gender;
Adaleigh - F
Execute : INSERT INTO seankkang.name_gender VALUES ('Adaleigh', 'F');
Amryn - Unisex
Execute : INSERT INTO seankkang.name_gender VALUES ('Amryn', 'Unisex');
Apurva - Unisex
Execute : INSERT INTO seankkang.name_gender VALUES ('Apurva', 'Unisex');
Aryion - M
Execute : INSERT INTO seankkang.name_gender VALUES ('Aryion', 'M');
Alixia - F
Execute : INSERT INTO seankkang.name_gender VALUES ('Alixia', 'F');
Alyssarose - F
Execute : INSERT INTO seankkang.name_gender VALUES ('Alyssarose', 'F');
Arvell - M
Execute : INSERT INTO seankkang.name_gender VALUES ('Arvell', 'M');
Aibel - M
Execute : INSERT INTO seankkang.name_gender VALUES ('Aibel', 'M');
Atiyyah - F
Execute : INSERT INTO seankkang.name_gender VALUES ('Atiyyah', 'F');
Adlie - F
Execute : INSERT INTO seankkang.name_gender VALUES ('Adlie', 'F');
Anyely - F
Execute : INSERT INTO seankkang.name_gender VALUES ('Anyely', 'F');
Aamoni - F
Execute : INSERT INTO seank

In [13]:
%%sql
SELECT COUNT(1) FROM seankkang.name_gender

 * postgresql://...
1 rows affected.


count
100
